# Tutorial de Ray

Ray es una API para Python que provee funcionalidades para desarrollar y ejecutar aplicaciones distribuidas. Permite a los usuarios agilizar la ejecución de un codigo mediante la paralelización, es decir, que se encarga de que ciertos procesos se puedan ejecutar al mismo tiempo, y así evitar que los tiempos de espera sean mas extendidos. 

A continuación, veremos detalladamente algunos de los puntos mas esenciales de Ray.

## 1 - Paralelización (Funciones remotas)

Con Ray las funciones se convierten en "funciones remotas", y son llamadas `Tareas` (o `Task` por su nombre en ingles). Estas funciones pueden ser ejecutadas de forma paralela. Para ello añadiremos "@ray.remote" sobre aquellas funciones que queramos transformar en funciones remotas. Este es un ejemplo de como convertiriamos una funcion regular de Python en una "función remota".

```python
# Función regular de Python.
def funcion_regular():
    return 1

# Función remota de Ray.
@ray.remote
def funcion_remota():
    return 1
```

A pesar de ser casi identicas a simple vista, una función remota tiene ciertas diferencias en comparación a una funcion regular de Python.

1. **Llamada a la función:** La función regular se llama con `funcion_regular()`, mientras que la funcion remota es invocada con `funcion_remota.remote()`.

2. **Retornar valores:** `funcion_regular` se ejecuta sincrónicamente y retorna el valor de la función (`1`), mientras que `funcion_remota` retorna inmediatamente un `IdObjeto` (un "Futuro") y ejecuta la tarea en segundo plano. El resultado del Futuro retornado se puede obtener llamando `ray.get` on the `ObjectID`.

    ```python
    >>> funcion_regular()
    1  # La funcion regular retorna directamente el resultado de la función
    
    >>> funcion_remota.remote()
    IdObjeto(1c80d6937802cd7786ad25e50caf2f023c95e350) # La funcion remota devuelve el ID del resultado futuro
    
    >>> ray.get(funcion_remota.remote()) # Obtenemos el resultado haciendo ray.get sobre el ID
    1
    ```
    
3. **Paralelismo:** Las ejecuciones de `funcion_regular` son **en serie**:

    ```python
    # Estas instrucciones son ejecutadas una por una.
    result = 0
    for _ in range(4):
        result += funcion_regular()
    assert result == 4
    ```
    Mientras que las ejecuciones de `funcion_remota` ocurren en **paralelo**:
    
    ```python
    # Estas instrucciones son ejecutadas al mismo tiempo en segundo plano, y los resultados son obtenidos por ray.get.
    results = []
    for _ in range(4):
        results.append(funcion_remota.remote())
    assert sum(ray.get(results)) == 4
     ```
  
  
**Ejemplo:** A continuación veremos un ejemplo de código para ver como transformaríamos una funcion regular, en una funcion remota, y como esto mejora notoriamente los tiempos de ejecución.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import ray
import time

print('Ray importado con exito!')

Ray importado con exito!


Por defecto, Ray no ejecuta mas tareas al mismo tiempo que el numero de CPUs, pero este ejemplo requiere que se ejecuten cuatro tareas al mismo tiempo, por lo que le decimos a Ray que hay 4 CPUs.

In [2]:
ray.init(num_cpus=4, ignore_reinit_error=True)

# Aplicamos la funcion Sleep para mejorar la precision de las mediciones de tiempo que se utilizarán, porque es posible
# que algunos procesos todavia se esten iniciando en segundo plano.
time.sleep(2.0)

2021-04-14 23:57:37,669	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


In [3]:
# Esta es una función que simula un trabajo mas "pesado", que tomaría 1 segundo en ejecutar
def funcion_reg(i):
    time.sleep(1)
    return i

start_time = time.time()

resultados = []
for i in range(4):
  resultados.append(funcion_reg(i))
  
duration = time.time() - start_time # Calculamos el tiempo que tomó en ejecutar esta tarea
print('Ejecutar esta iteración tomó {:.3f} segundos.'.format(duration))
print('Los resultados son:', resultados)

Ejecutar esta iteración tomó 4.029 segundos.
Los resultados son: [0, 1, 2, 3]


Ahora veremos como se comporta la misma función, pero si le aplicamos Ray:

In [4]:
@ray.remote  # Agregamos el @ray.remote para indicar que es una funcion remota
def funcion_rem(i):
    time.sleep(1)
    return i

start_time = time.time()

resultados = []
for i in range(4):
  resultados.append(funcion_rem.remote(i)) # En esta linea estamos almacenando en el arreglo "resultados" los ID de los
                                          # objetos (futuros) retornados por la funcion remota (la cual es llamada con .remote)

resultados = ray.get(resultados) # Obtenemos el valor de los futuros aplicando el ray.get a los id oobtenidos previamente
duration = time.time() - start_time # Calculamos el tiempo de ejecución

print('Ejecutar esta iteracion tomó {:.3f} segundos.'.format(duration))
print('Los resultados son:', resultados)

Ejecutar esta iteracion tomó 1.137 segundos.
Los resultados son: [0, 1, 2, 3]


**Resultado:** Se puede apreciar como con unos pocos cambios de codigo, la ejecución de la función remota fue mas rápida que la función regular, ya que al momento de ejecutar el loop "for", Ray realizó las 4 iteraciones en paralelo.

## 2 - Procesamiento de datos en paralelo

La siguiente función remota, retorna el mismo argumento que se le pasa. Si le pasamos un objeto, el resultado obtenido por ray.get debería ser el mismo objeto.

```python
@ray.remote
def f(x): # Le pasamos un objeto por parametro
    return x # Retornamos el mismo objeto

>>> x1_id = f.remote(1) # El valor retornado por la función es el ID del objeto 
>>> ray.get(x1_id) # obtenemos su valor utilizando el ray.get
1

>>> x2_id = f.remote([1, 2, 3])
>>> ray.get(x2_id)
[1, 2, 3]
```

Tambien podemos pasar el ID de algun objeto por parametro en una funcion remota. Cuando se ejecute la funcion, Ray automaticamente convertira ese ID en el objeto al que esta referenciando.

```python
>>> y1_id = f.remote(x1_id) # Le pasamos el id del objeto por parametro y lo almacenamos dentro de una variable 
>>> ray.get(y1_id) # Al hacer ray.get obetenemos el objeto referenciado
1

>>> y2_id = f.remote(x2_id)
>>> ray.get(y2_id)
[1, 2, 3]
```

En este ultimo ejemplo, la tarea que crea `y1_id` no se ejecutará hasta que finalice de ejecutarse la tarea que crea `x1_id`.

**Ejemplo:** Ahora veremos un ejemplo 

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import time

import ray

In [6]:
ray.init(num_cpus=4, ignore_reinit_error=True)

time.sleep(2.0)

2021-04-14 23:58:07,706	INFO worker.py:667 -- Calling ray.init() again after it has already been called.


In [7]:
# Estas funciones simulan procesos mas complicados, usando el time.sleep para que tarden mas en ejecutar.
def load_data(filename):
    time.sleep(0.1)
    return np.ones((1000, 100))

def normalize_data(data):
    time.sleep(0.1)
    return data - np.mean(data, axis=0)

def extract_features(normalized_data):
    time.sleep(0.1)
    return np.hstack([normalized_data, normalized_data ** 2])

def compute_loss(features):
    num_data, dim = features.shape
    time.sleep(0.1)
    return np.sum((np.dot(features, np.ones(dim)) - np.ones(num_data)) ** 2)

In [8]:
start_time = time.time()
losses = []
for filename in ['file1', 'file2', 'file3', 'file4']:
    inner_start = time.time()

    data = load_data(filename)
    normalized_data = normalize_data(data)
    features = extract_features(normalized_data)
    loss = compute_loss(features)
    losses.append(loss)
    
    inner_end = time.time()
    

print('Las perdidas son {}.'.format(losses) + '\n')
loss = sum(losses)

duration = time.time() - start_time

print('La perdida total es {}. Esto tardó {:.3f} segundos.'.format(loss, duration))

Las perdidas son [1000.0, 1000.0, 1000.0, 1000.0].

La perdida total es 4000.0. Esto tardó 1.833 segundos.


**Comparación:** Ahora probemos el mismo ejemplo, pero esta vez utilizando Ray, y veamos como afecta en el tiempo de ejecución

In [9]:
@ray.remote # Aplicamos el ray.remote sobre todas las funciones para convertirlas en funciones remotas
def load_data(filename):
    time.sleep(0.1)
    return np.ones((1000, 100))

@ray.remote
def normalize_data(data):
    time.sleep(0.1)
    return data - np.mean(data, axis=0)

@ray.remote
def extract_features(normalized_data):
    time.sleep(0.1)
    return np.hstack([normalized_data, normalized_data ** 2])

@ray.remote
def compute_loss(features):
    num_data, dim = features.shape
    time.sleep(0.1)
    return np.sum((np.dot(features, np.ones(dim)) - np.ones(num_data)) ** 2)

In [10]:
start_time = time.time()
losses = []
for filename in ['file1', 'file2', 'file3', 'file4']:
    inner_start = time.time()

    data = load_data.remote(filename)
    normalized_data = normalize_data.remote(data)
    features = extract_features.remote(normalized_data)
    loss = compute_loss.remote(features)
    losses.append(loss) 
    
    inner_end = time.time()
    
        
losses = ray.get(losses) # Obtenemos los objetos haciendo ray.get sobre los IDs
print('Las perdidas son {}.'.format(losses) + '\n')
loss = sum(losses)

duration = time.time() - start_time

print('La perdida total es {}. Esto tardó {:.3f} segundos.'.format(loss, duration))

Las perdidas son [1000.0, 1000.0, 1000.0, 1000.0].

La perdida total es 4000.0. Esto tardó 0.478 segundos.


## 3 - Paralelismo anidado

En este punto veremos como crear tareas anidadas, llamando una funcion remota dentro de otra funcion remota. También veremos unos ejemplos para apreciar mejor como se reducen los tiempos de ejecución.

Al igual que las funciones regulares, las funciones remotas pueden llamar a otras funciones remotas, como se muestra en el siguiente ejemplo:

```python
@ray.remote
def f():
    return 1

@ray.remote
def g():
    # Llama a la funcion f 4 veces y retorna el arreglo con los IDs de los objetos.
    results = []
    for _ in range(4):
      results.append(f.remote())
    return results

@ray.remote
def h():
    # Llama a f 4 veces y antes de retornar el resultado debe esperar a que finalicen las 4 tareas,
    # entonces retornará un arreglo con los objetos.
    results = []
    for _ in range(4):
      results.append(f.remote())
    return ray.get(results)
```

Las funciones g() y h() retornarán los siguientes valores

```python

>>> ray.get(g.remote()) # g retorna un arreglo con los IDs de los objetos, ya que al llamar a f no se aplica ray.remote
[ObjectID(b1457ba0911ae84989aae86f89409e953dd9a80e),
 ObjectID(7c14a1d13a56d8dc01e800761a66f09201104275),
 ObjectID(99763728ffc1a2c0766a2000ebabded52514e9a6),
 ObjectID(9c2f372e1933b04b2936bb6f58161285829b9914)]

>>> ray.get(h.remote()) # h retorna el arreglo con los objetos que retorna f
[1, 1, 1, 1]
```

**Ejemplo:** Ahora veremos un codigo ejecutable para apreciar de mejor manera como se paraleliza un codigo con funciones anidadas.

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import time

import ray

In [12]:
ray.init(num_cpus=9, ignore_reinit_error=True)

time.sleep(2.0)

2021-04-15 00:00:09,634	INFO worker.py:667 -- Calling ray.init() again after it has already been called.


In [13]:
def compute_gradient(data, current_model):
    time.sleep(0.03)
    return 1

def train_model(hyperparameters):
    current_model = 0
    for _ in range(10):
        gradients = []
        for j in range(2):
            gradients.append(compute_gradient(j, current_model))
        current_model += sum(gradients)

    return current_model


In [14]:
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

# Run some hyperparaameter experiments.
results = []
for hyperparameters in [{'learning_rate': 1e-1, 'batch_size': 100},
                        {'learning_rate': 1e-2, 'batch_size': 100},
                        {'learning_rate': 1e-3, 'batch_size': 100}]:
    results.append(train_model(hyperparameters))

end_time = time.time()
duration = end_time - start_time

print('El resultado es {}. Esto tardó {:.3f} segundos.'.format(results, duration))

El resultado es [20, 20, 20]. Esto tardó 1.912 segundos.


**Comparación:** Veamos ahora como se comportaría este mismo codigo si le aplicamos Ray.

In [19]:
@ray.remote
def compute_gradient(data, current_model):
    time.sleep(0.03)
    return 1

@ray.remote
def train_model(hyperparameters):
    current_model = 0
    for _ in range(10):
        gradients = []
        for j in range(2):
            gradients.append(compute_gradient.remote(j, current_model))
        current_model += sum(gradients)

    return current_model

In [20]:
# Sleep a little to improve the accuracy of the timing measurements below.
time.sleep(2.0)
start_time = time.time()

# Run some hyperparaameter experiments.
results = []
for hyperparameters in [{'learning_rate': 1e-1, 'batch_size': 100},
                        {'learning_rate': 1e-2, 'batch_size': 100},
                        {'learning_rate': 1e-3, 'batch_size': 100}]:
    results.append(train_model.remote(hyperparameters))


results = ray.get(results)
end_time = time.time()
duration = end_time - start_time

print('El resultado es {}. Esto tardó {:.3f} segundos.'.format(results, duration))

El resultado es [20, 20, 20]. Esto tardó 2.515 segundos.


## 4 - Actores

En Ray, un Actor es muy parecido a lo que es una clase normal de Python. Se inicializa de la misma manera (con la función `__init__` ) y puede contener funciones internas, a las que se deben acceder con los metodos remotos.

Para crear un Actor solo debemos colocar el `@ray.remote` sobre una clase, como se muestra en el siguiente ejemplo:

```python
@ray.remote
class Example(object):
    def __init__(self, x):
        self.x = x
    
    def set(self, x):
        self.x = x
    
    def get(self):
        return self.x
```
Un Actor se diferencia de una clase regular de Python en algunos puntos muy especificos que se presentan a continuación:

1. **Instanciación:** Una clase regular de Python es instanciada con: `e = Example(1)` 

    Mientras que un Actor se instancia con el metodo remoto:  `e = Example.remote(1)`
    

2. **Invocación de metodos:** Los metodos de una clase regular serían invocados con `e.set(2)` o `e.get()`, mientras que los metodos de un Actor serían invocados de la siguiente manera:

    ```python
        >>> e.set.remote(2)
        ObjectID(d966aa9b6486331dc2257522734a69ff603e5a1c)
    
        >>> e.get.remote()
        ObjectID(7c432c085864ed4c7c18cf112377a608676afbc3)
    ```

3. **Retornar valores:** Los metodos de un Actor retornar de manera inmediata el ID del objeto resultante. Este resultado se puede obtener mediante el `ray.get`.

    ```python
        >>> ray.get(e.set.remote(2))
        None
    
        >>> ray.get(e.get.remote())
        2
    ```

**Ejemplo:** Ahora veremos un ejemplo de como aplicar los Actores de Ray:

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from collections import defaultdict
import numpy as np
import time

import ray

print('Ray importado con exito!')

Ray importado con exito!


In [3]:
ray.init(num_cpus=4, ignore_reinit_error=True)

2021-04-16 04:30:31,696	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.104',
 'raylet_ip_address': '192.168.0.104',
 'redis_address': '192.168.0.104:6379',
 'object_store_address': 'tcp://127.0.0.1:64623',
 'raylet_socket_name': 'tcp://127.0.0.1:58399',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\vicen\\AppData\\Local\\Temp\\ray\\session_2021-04-16_04-30-28_385141_996',
 'metrics_export_port': 64469,
 'node_id': '1bd7eed63683c87de73446c4cae35011150385da'}

In [10]:
class Foo(object):
    def __init__(self):
        self.counter = 0

    def reset(self):
        self.counter = 0

    def increment(self):
        time.sleep(0.5)
        self.counter += 1
        return self.counter

In [11]:
# Creamos dos objetos de la clase Foo
f1 = Foo()
f2 = Foo()

(pid=3860) Windows fatal exception: access violation
(pid=3860) 
(pid=1292) Windows fatal exception: access violation
(pid=1292) 


In [12]:
start_time = time.time()

f1.reset()
f2.reset()

results = []
for _ in range(5): # Incrementaremos 1 en 1 el estado de los objetos hasta llegar a 5.
    results.append(f1.increment())
    results.append(f2.increment())

duration = time.time() - start_time # Calculamos el tiempo de ejecución de este ejemplo.

print('Los resultados son: {}. Esto tardó {:.3f} segundos'.format(results, duration))

Los resultados son: [1, 1, 2, 2, 3, 3, 4, 4, 5, 5]. Esto tardó 5.087 segundos


**Comparación:** Ahora veremos como funciona si usamos actores en lugar de clases

In [7]:
@ray.remote
class Foo(object):
    def __init__(self):
        self.counter = 0

    def reset(self):
        self.counter = 0

    def increment(self):
        time.sleep(0.5)
        self.counter += 1
        return self.counter

In [8]:
# Creamos dos actores
f1 = Foo.remote()
f2 = Foo.remote()

In [9]:
start_time = time.time()

# Reiniciamos el estado del actor y asi podemos correr esta celda multiples veces sin cambiar los resultados.
f1.reset.remote()
f2.reset.remote()

results = []
for _ in range(5):
    results.append(f1.increment.remote()) # Llenamos el arreglo results con los ID retornados por los metodos del actor
    results.append(f2.increment.remote())    
    
results = ray.get(results) # Obtenemos el valor de los resultados retornados por los metodos del Actor
                           # usando ray.get

duration = time.time() - start_time

print('Los resultados son: {}. Esto tardó {:.3f} segundos'.format(results, duration))

Los resultados son: [1, 1, 2, 2, 3, 3, 4, 4, 5, 5]. Esto tardó 2.736 segundos


## 5 - Compartir referencias a un Actor

Supongamos que en algun momento queramos tener varias tareas remotas que invocan metodos en el mismo actor. Para esto, podemos pasar el identificador del Actor como un argumento para las tareas.

Primero creamos el actor de la siguiente manera:

```python
@ray.remote
class Actor(object):
    def method(self):
        pass

# Creamos el actor
actor = Actor.remote()
```
Luego, definimos una funcion remota que reciba un identificador del actor como argumento


```python
@ray.remote
def f(actor):
    # Podemos invocar un metodo en el actor y esperar sus resultados
    return ray.get(actor.method.remote())
```

Despues podemos invocar la funcion remota varias veces y pasarle el identificador del actor como argumento

```python
# Cada una de estas funciones remotas invocará metodos en el mismo actor.
f.remote(actor)
f.remote(actor)
f.remote(actor)
```

**Ejemplo:** Ahora veremos un ejemplo de como compartir referencias usando actores

In [34]:
@ray.remote
class LoggingActor(object):
    def __init__(self):
        self.logs = defaultdict(lambda: [])
    
    def log(self, index, message):
        self.logs[index].append(message)
    
    def get_logs(self):
        return dict(self.logs)


In [35]:
logging_actor = LoggingActor.remote()

In [36]:
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(60):
        time.sleep(1)
        # Push a logging message to the actor.
        logging_actor.log.remote(experiment_index, 'On iteration {}'.format(i))

In [37]:
experiment_ids = [run_experiment.remote(i, logging_actor) for i in range(3)]

In [42]:
logs = logging_actor.get_logs.remote()
logs = ray.get(logs)

logs

{0: ['On iteration 0',
  'On iteration 1',
  'On iteration 2',
  'On iteration 3',
  'On iteration 4',
  'On iteration 5',
  'On iteration 6',
  'On iteration 7',
  'On iteration 8',
  'On iteration 9',
  'On iteration 10',
  'On iteration 11',
  'On iteration 12',
  'On iteration 13',
  'On iteration 14',
  'On iteration 15',
  'On iteration 16',
  'On iteration 17',
  'On iteration 18',
  'On iteration 19',
  'On iteration 20',
  'On iteration 21',
  'On iteration 22',
  'On iteration 23',
  'On iteration 24',
  'On iteration 25',
  'On iteration 26',
  'On iteration 27',
  'On iteration 28',
  'On iteration 29',
  'On iteration 30',
  'On iteration 31',
  'On iteration 32',
  'On iteration 33',
  'On iteration 34',
  'On iteration 35',
  'On iteration 36',
  'On iteration 37',
  'On iteration 38',
  'On iteration 39',
  'On iteration 40',
  'On iteration 41',
  'On iteration 42',
  'On iteration 43',
  'On iteration 44',
  'On iteration 45',
  'On iteration 46',
  'On iteration 47',

**Resultado:** Prueba ejecutando la ultima instruccion reiteradas veces y mira como cambian los resultados de salida.

## 6 - Ray.wait

Luego de ejecutar un numero de tareas, en ciertos casos querrás saber cuales fueron las que terminaron primero en ejecutar. Esto se puede resolver usando `ray.wait`. Esta funcion se utiliza de la siguiente manera:

    ids_listos, ids_restantes = ray.wait(ids_totales, num_retornos = 1, timeout = None)
    
Donde

**Argumentos:**
- ids_totales = Es una lista de ids de objetos.
- num_retornos = Este es el numero maximo de IDs a esperar que terminen.
- timeout = Esta es la cantidad de tiempo maxima (en milisegundos) a esperar. Ray.wait esperará hasta que los num_retornos objetos esten listos, o hasta que el tiempo estipulado en timeout haya pasado.

**Valores retornados:**
- ids_listos = Esta es una lista de objetos que estan disponibles.
- ids_restantes = Es la lista de objetos que que restan de los ids_totales.

En pocas palabras, a este metodo le debemos pasar como argumento una lista de ids, y ray.wait guardará en ids_listos todos aquellos objetos que ya esten disponibles, y en ids_restantes todos los que van quedando.

**Ejemplo:** Ahora veremos un ejemplo de como aplicar ray.wait para cambiar los resultados a obtener.

In [43]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

print('Successfully imported ray!')

Successfully imported ray!


In [44]:
ray.init(num_cpus=6, ignore_reinit_error=True)

# Sleep a little to improve the accuracy of the timing measurements used below,
# because some workers may still be starting up in the background.
time.sleep(2.0)

2021-04-16 05:32:19,521	INFO worker.py:667 -- Calling ray.init() again after it has already been called.


In [49]:
@ray.remote
def f(i):
    np.random.seed(5 + i)
    x = np.random.uniform(0, 4)
    time.sleep(x)
    return i, time.time()

In [50]:
start_time = time.time()

# This launches 6 tasks, each of which takes a random amount of time to
# complete.
result_ids = [f.remote(i) for i in range(6)]
# Get one batch of tasks. Instead of waiting for a fixed subset of tasks, we
# should instead use the first 3 tasks that finish.
initial_results = ray.get(result_ids[:3])

print(initial_results)

end_time = time.time()
duration = end_time - start_time

[(0, 1618565606.741766), (1, 1618565609.4293647), (2, 1618565606.1633282)]


In [51]:
remaining_results = ray.get(result_ids[3:])
print(remaining_results)

[(3, 1618565609.6735802), (4, 1618565606.7887244), (5, 1618565609.8779786)]


In [52]:
initial_indices = [result[0] for result in initial_results]
initial_times = [result[1] for result in initial_results]
remaining_indices = [result[0] for result in remaining_results]
remaining_times = [result[1] for result in remaining_results]

print('Este ejemplo tardó {} segundos.'.format(duration))

Success! The example took 3.6112260818481445 seconds.


Ahora el mismo codigo, pero utilizando la funcion ray.wait

In [53]:
@ray.remote
def f(i):
    np.random.seed(5 + i)
    x = np.random.uniform(0, 4)
    time.sleep(x)
    return i, time.time()

In [54]:
start_time = time.time()

# This launches 6 tasks, each of which takes a random amount of time to
# complete.
result_ids = [f.remote(i) for i in range(6)]
# Get one batch of tasks. Instead of waiting for a fixed subset of tasks, we
# should instead use the first 3 tasks that finish.
initial_results, remaining_results = ray.wait(result_ids, num_returns = 3, timeout = None)
initial_results = ray.get(initial_results)

print(initial_results)

end_time = time.time()
duration = end_time - start_time

[(0, 1618565879.3418486), (2, 1618565878.7677207), (4, 1618565879.3895175)]


In [55]:
remaining_results = ray.get(remaining_results)
print(remaining_results)

[(1, 1618565882.025153), (3, 1618565882.2723637), (5, 1618565882.491053)]


In [56]:
initial_indices = [result[0] for result in initial_results]
initial_times = [result[1] for result in initial_results]
remaining_indices = [result[0] for result in remaining_results]
remaining_times = [result[1] for result in remaining_results]

print('Este ejemplo tardó {} segundos.'.format(duration))

Este ejemplo tardó 0.9705512523651123 segundos.


## 7 - Procesar tareas en orden de termino

Ahora veremos un ejemplo de como utilizar ray.wait para procesar tareas en el orden en que terminan.
Primero veamos un codigo sin ray.wait y veamos como procesa sus tareas:

In [65]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import ray
import time

In [66]:
ray.init(num_cpus=5,  ignore_reinit_error=True)

2021-04-16 05:48:18,889	INFO worker.py:667 -- Calling ray.init() again after it has already been called.


In [67]:
@ray.remote
def f():
    time.sleep(np.random.uniform(0, 5))
    return time.time()

In [68]:
start_time = time.time()

remaining_result_ids = [f.remote() for _ in range(10)]

# Get the results.
results = []
while len(remaining_result_ids) > 0:
    # EXERCISE: Instead of simply waiting for the first result from
    # remaining_result_ids, use ray.wait to get the first one to finish.
    result_id = remaining_result_ids[0]
    remaining_result_ids = remaining_result_ids[1:]
    result = ray.get(result_id)
    results.append(result)
    print('Este procesamiento terminó en {} segundos.'
          .format(result - start_time))    

end_time = time.time()
duration = end_time - start_time

Processing result which finished after 0.884099006652832 seconds.
Processing result which finished after 2.6905689239501953 seconds.
Processing result which finished after 4.868346214294434 seconds.
Processing result which finished after 1.3337371349334717 seconds.
Processing result which finished after 4.77510404586792 seconds.
Processing result which finished after 4.944920063018799 seconds.
Processing result which finished after 9.580554962158203 seconds.
Processing result which finished after 9.549487113952637 seconds.
Processing result which finished after 9.757725954055786 seconds.
Processing result which finished after 7.068840980529785 seconds.


In [69]:
print('Este ejemplo tardó {} segundos.'.format(duration))

Este ejemplo tardó 9.758952856063843 segundos.


En este ejemplo de arriba podemos ver como los resultados fueron procesados en el orden que son ejecutados, y no en el orden en que terminaron. Esto se puede cambiar utilizando ray.wait de la siguiente forma:

In [70]:
@ray.remote
def f():
    time.sleep(np.random.uniform(0, 5))
    return time.time()

In [71]:
start_time = time.time()

remaining_result_ids = [f.remote() for _ in range(10)]

# Get the results.
results = []
while len(remaining_result_ids) > 0:
    # EXERCISE: Instead of simply waiting for the first result from
    # remaining_result_ids, use ray.wait to get the first one to finish.
    result_id, remaining_result_ids = ray.wait(remaining_result_ids, num_returns = 1, timeout = None)
    result = ray.get(result_id)[0]
    results.append(result)
    print('Este procesamiento terminó en {} segundos.'
          .format(result - start_time))    

end_time = time.time()
duration = end_time - start_time

Processing result which finished after 0.04191732406616211 seconds.
Processing result which finished after 2.5301694869995117 seconds.
Processing result which finished after 2.6169867515563965 seconds.
Processing result which finished after 4.4124579429626465 seconds.
Processing result which finished after 6.3186116218566895 seconds.
Processing result which finished after 6.681273698806763 seconds.
Processing result which finished after 7.188187837600708 seconds.
Processing result which finished after 8.670400381088257 seconds.
Processing result which finished after 9.754715204238892 seconds.
Processing result which finished after 10.377838373184204 seconds.


In [72]:
print('Este ejemplo tardó {} segundos.'.format(duration))

Este ejemplo tardó 10.377838373184204 segundos.


Podemos ver ahora como los resutaldos fueron procesados en el orden en que finalizaron.